# Downloading data

In [ ]:
#mkdir train valid test 

def download(url, destination_folder='.'):
  !wget -nc -q --show-progress $url -P $destination_folder

phases = ['train', 'valid', 'test']
for phase in phases:
  download(f'https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/{phase}.zip', phase)

In [ ]:
import zipfile

from tqdm import tqdm_notebook as tqdm

for phase in phases:
  with zipfile.ZipFile(os.path.join(phase, f'{phase}.zip'), 'r') as myzip:
    for file in tqdm(myzip.namelist(), desc=f'Extracting {phase}.zip'):
      myzip.extract(member=file, path=phase)

In [ ]:
phases = ['train', 'valid', 'test']

data_root = os.path.join(os.getcwd())
data_dir = {phase:os.path.join(data_root, phase, phase) for phase in phases}

In [ ]:
#data_dir['test']
data_dir

In [ ]:
# There are three different classes

import glob
classes = ['melanoma', 'nevus', 'seborrheic_keratosis']
classes

In [ ]:
import seaborn as sns
import warnings
import pandas as pd
import numpy as np

def print_images_distribution(plot=False):
  
  image_repartition = pd.DataFrame(index=[d.split(os.path.sep)[-1] for d in data_dir.values()], columns=classes)
  plot_data = pd.DataFrame(columns=['Class', 'Phase', 'Count'])
  
  for phase in [d.split(os.path.sep)[-1] for d in data_dir.values()]:
    for disease in classes:
      count =  len(glob.glob(os.path.join(data_root, phase, phase, disease, '*.jpg')))
      image_repartition.loc[phase][disease] = count
      plot_data = plot_data.append({'Class': disease, 'Phase': phase, 'Count': count}, ignore_index=True)

  image_repartition.loc['TOTAL'] = image_repartition.sum(axis=0)          
  image_repartition['TOTAL'] = image_repartition.sum(axis=1).astype(int)
  image_repartition['Ratio'] = np.round(image_repartition.TOTAL / (image_repartition.TOTAL.sum() - image_repartition.TOTAL[-1]), 2)
  
  if plot:
    sns.set_style("whitegrid")
    sns.barplot(x='Class', y='Count', hue='Phase', data=plot_data, palette='Blues')

  return image_repartition

In [ ]:
print_images_distribution(True)